In [1]:
%load_ext autoreload
%autoreload 2

import time
import random
import pickle, os
import numpy as np
import cvxpy as cp

from copy import deepcopy

import matplotlib.pyplot as plt
%matplotlib inline

import torch
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter

from free_flyer.free_flyer import FreeFlyer
from free_flyer.utils import *
from solvers.coco_ff import CoCo_FF
from solvers.meta_ff import Meta_FF

In [89]:
#load train/test data
prob = FreeFlyer() #use default config, pass different config file oth.
config_fn = './free_flyer/config/default.p'

config_file = open(config_fn,'rb')
dataset_name, _, _ = pickle.load(config_file); config_file.close()

relative_path = os.getcwd()
dataset_fn = relative_path + '/free_flyer/data/' + dataset_name

train_file = open(dataset_fn+'/train.p','rb')
# p_train, x_train, u_train, y_train, c_train, times_train = pickle.load(train_file)
train_data = pickle.load(train_file)
train_file.close()
x_train = train_data[1]
y_train = train_data[3]

test_file = open(dataset_fn+'/test.p','rb')
# p_test, x_test, u_test, y_test, c_test, times_test = pickle.load(test_file)
test_data = pickle.load(test_file)
p_test, x_test, u_test, y_test, c_test, times_test = test_data
test_file.close()

n_test = x_test.shape[0]

In [110]:
system = 'free_flyer'
prob_features = ['x0', 'obstacles_map']

meta_learner = Meta_FF(system, prob, prob_features)

n_features = 4
device_id= 1
meta_learner.construct_strategies(n_features, train_data, device_id=device_id)

In [86]:
fn_saved = 'models/CoCoFF_free_flyer_20210125_0808.pt'
meta_learner.load_network(fn_saved, False)

meta_learner.model_fn

Loading presaved classifier model from models/CoCoFF_free_flyer_20210125_0808.pt


'/home/acauligi/cs_234/project/mlopt-micp/MetaCoCoFF_free_flyer_20210330_1243.pt'

In [112]:
meta_learner.model.ff_shape

[128, 128, 128, 15]

In [111]:
meta_learner.train(train_data, "runs/asdfasdf")
# meta_learner.finetuning(test_data, "runs/finetunning")

Got to iteration 1

fast weights sum: -1.5312278270721436

Got to iteration 2

fast weights sum: -2.470101833343506

Got to iteration 3

fast weights sum: -3.513078451156616

Got to iteration 4

fast weights sum: -4.54121208190918

Got to iteration 5

fast weights sum: -5.595929145812988

Got to iteration 6

fast weights sum: -6.499002933502197

Got to iteration 7

fast weights sum: -7.203591346740723

Got to iteration 8

fast weights sum: -7.694435119628906

Got to iteration 9

fast weights sum: -7.914017677307129

Got to iteration 10

fast weights sum: -7.936307430267334

Got to iteration 11

fast weights sum: -7.895885467529297

Got to iteration 12

fast weights sum: -7.826658725738525

Got to iteration 13

fast weights sum: -7.788736343383789

Got to iteration 14

fast weights sum: -7.707894802093506

Got to iteration 15

fast weights sum: -7.580371379852295

Got to iteration 16

fast weights sum: -7.529297351837158

Got to iteration 17

fast weights sum: -7.6240034103393555

Got t

KeyboardInterrupt: 

In [18]:
for ii in range(len(meta_learner.model.vars)):
    print(meta_learner.model.vars[ii].requires_grad)

True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
coco_last_layer = list(meta_learner.model.parameters())[-2:]
shared_params = list(meta_learner.model.parameters())[:-2]
len(list(shared_params + coco_last_layer))

In [ ]:
# def train_meta(update_lr, meta_lr, device_id):
#     system = 'free_flyer'
#     prob_features = ['x0', 'obstacles_map']

#     meta_learner = Meta(system, prob, prob_features)

#     n_features = 4
#     meta_learner.construct_strategies(n_features, train_data, device_id=device_id)

#     fn_saved = 'models/mloptff_free_flyer_20201103_1353.pt'
#     meta_learner.load_network(fn_saved, False)
#     print('meta_learner fn: {}'.format(meta_learner.model_fn))

#     print('Meta lr: {}'.format(meta_lr))
#     print('Update lr: {}'.format(update_lr))
#     meta_learner.update_lr = update_lr
#     meta_learner.meta_lr = meta_lr
    
#     writer_fn = 'runs/update_{}_meta_{}'.format(int(round(-100.*np.log10(meta_learner.update_lr))), int(round(-100.*np.log10(meta_learner.meta_lr))))

#     meta_learner.train(train_data, writer_fn, verbose=True)

In [ ]:
# # import multiprocessing

# # processes = []
# # for tup in tups:
# #     update_lr, meta_lr, device_id = tup
# #     p = multiprocessing.Process(target=train_meta, args=[update_lr, meta_lr, device_id])

# #     p.start()
# #     processes.append(p)
    
# # for p in processes:
# #     p.join()

# ctr = 1
# device_id = 1
# for meta_lr in [5e-4, 1e-4, 5e-5]:
#     for update_lr in [1e-5, 5e-6, 1e-6]:
#         dump_fn = './training_logs/training_{}.txt'.format(ctr)
#         file = open(dump_fn,'w')
#         file.write('On meta_lr {}\n'.format(meta_lr))
#         file.write('On update_lr {}\n'.format(update_lr))
#         file.close()
#         train_meta(update_lr, meta_lr, device_id)
#         dump_fn = './training_logs/training_{}.txt'.format(ctr)
#         file = open(dump_fn,'w')
#         file.write('On meta_lr {}\n'.format(meta_lr))
#         file.write('On update_lr {}\n'.format(update_lr))
#         file.write('Done')
#         file.close()
#         ctr += 1